# Simple 1D fault slip modeling

Gareth Funning, University of California, Riverside

To demonstrate some very simple features of nonlinear models, I will use one of the very simplest models out there $-$ the 1D model for strike-slip, as popularized by [Savage and Burford (1973)](https://doi.org/10.1029/JB078i005p00832).

## 1. Dependencies 

If you don't have these things installed, you're probably in the wrong conda environment!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 2. The 1D model for strike-slip

Although it is most commonly known as the Savage and Burford model, the 1D model for strike-slip, or more precisely, for a screw dislocation in an elastic half space, was originally derived by Weertman and Weertman (1964). It makes several simplifying assumptions:

- The fault is vertical
- The fault is infinitely long
- The fault extends infinitely deep, beneath a locking depth, $D$
- The fault is locked above the locking depth
- The fault is stably sliding at a constant, uniform rate, $s$, beneath the locking depth

The model describes how the surface deformation rate, $v$, varies as a function of distance, $x$, from the fault. (There are also expressions for strain rate, $e$, which are effectively the spatial derivative of v, but we won't get into that today.) That relationship is analytical, and simple to evaluate:

$$ v = \frac{s}{\pi} \arctan\left(\frac{x}{D}\right)+v_{shift}$$,

where $v_{shift}$ is a shift in velocity to account for the difference between zero in the model and any overall regional motion (e.g. due to plate tectonics).

The model is often associated with Savage and Burford, two researchers from the USGS, who in 1973 published their application of the model to EDM data from the San Andreas fault system in California. The main figure illustrating the model is a classic, and worth reproducing here!

![The 1D model from Savage and Burford, 1973](savage_burford_1973.png)

(Savage and Burford, 1973)

The top panel shows the model geometry (left), the theoretical strain rate distribution (center) and expected displacement (or velocity) profile (right). The lower panel shows the same, but for a fault that is stably sliding from the surface (what you would expect if a fault was creeping at its long-term slip rate).


## 3. Evaluating the 1D model

As the model is simple, we can evaluate it easily in Python. We can try to see what happens when we vary the input parameters.

First, we can make a function to evaluate the model to make things simpler...

In [ ]:
# let's define a function for the 1d model:
def slip1d(s, D, v_shift, x):
    v=(s/np.pi)*np.arctan(x/D)+v_shift
    return v

# let's establish some parameter/input values

s=30        # slip rate in mm/yr
D=15        # locking depth in km
v_shift=0   # velocity shift in mm/yr

x=np.linspace(-100,100,101)  # an array of distances in km

# and evaluate it!

v=slip1d(s,D,v_shift,x)

# let's plot it to see what we got

plt.plot(x, v)  # Plot the chart
plt.xlabel("distance from fault (km)")  
plt.ylabel("fault-parallel velocity (mm/yr)")  
plt.title("slip rate: {0:4.1f}, locking depth: {1:4.1f}, velocity shift:{2:4.1f}".format(s,D,v_shift))
plt.show()  # display


### 3.1 Varying slip rate

What happens if we vary slip rate? We can look though a few possible values and see what happens?

In [ ]:
# make an array of slip rate values
smin=5
smax=50
sstep=5
slip_rates=np.arange(smin,smax+sstep,sstep)

# keep everything else the same
D=15        # locking depth in km
v_shift=0   # velocity shift in mm/yr
x=np.linspace(-100,100,101)  # an array of distances in km

# and loop through all the slip rate values
for s in slip_rates:
    v=v=slip1d(s,D,v_shift,x)
    plt.plot(x, v)  # Plot the chart

    
plt.xlabel("distance from fault (km)")  
plt.ylabel("fault-parallel velocity (mm/yr)")  
plt.title("slip rate: {0:4.1f}-{1:4.1f}, locking depth: {2:4.1f}, velocity shift:{3:4.1f}"
          .format(smin,smax,D,v_shift))
plt.show()  # display    

How does the shape of the curves change as $s$ increases? (Maybe try restricting the range of $x$ values to accentuate it?) You should see that the curves show their maximum curvature at the same distance from the fault, regardless of the slip rate. This is because $s$ is a **linear parameter** of the model $-$ as $s$ increases, so does $v$, proportionately. 

## 3.2 Varying locking depth

Let's try varying the locking depth now. 

In [ ]:
# make an array of locking depth values
Dmin=5
Dmax=25
Dstep=4
locking_depths=np.arange(Dmin,Dmax+Dstep,Dstep)

# keep everything else the same
s=30        # locking depth in km
v_shift=0   # velocity shift in mm/yr
x=np.linspace(-100,100,101)  # an array of distances in km

# and loop through all the slip rate values
for D in locking_depths:
    v=v=slip1d(s,D,v_shift,x)
    plt.plot(x, v)  # Plot the chart

    
plt.xlabel("distance from fault (km)")  
plt.ylabel("fault-parallel velocity (mm/yr)")  
plt.title("slip rate: {0:4.1f}, locking depth: {1:4.1f}-{2:4.1f}, velocity shift:{3:4.1f}"
          .format(s,Dmin,Dmax,v_shift))
plt.show()  # display    

How does the shape of the curves change as $D$ increases? (Maybe try restricting the range of $x$ values to accentuate it?) You should see that the curves vary in how sharply they curve, and where. Smaller and smaller values of $D$ get closer and closer to a 'step' shape, with the sharp curves getting closer and closer to the location of the fault. Conversely, as $D$ gets larger, the point of maximum curvature gets further and further away from the fault. Note also the variation in the size of the velocities $-$ the curves all have different maximum velocities, although we didn't change the slip rate.

This is all because $D$ is a **nonlinear parameter** of the model $-$ as $D$ increases, both the shape and the size of $v$ change, but not in a proportionate manner. 

## 4. Solving for a real slip rate

We will load in some real GNSS velocities, campaign measurements from the 1990s, as a simple example of how to apply the 1D model to real data.

In [ ]:
# let's look at the data
!cat gps_data.txt

The columns represent:

- site_name 
- fault_parallel_velocity (mm/yr)
- fault_parallel_error (mm/yr)
- fault_normal_velocity (mm/yr)
- fault_normal_error (mm/yr)
- distance (mm/yr)

Loading them is simple (and we can exclude irrelevant things).

In [ ]:
gps_data=np.loadtxt('gps_data.txt',delimiter=' ',usecols=(1,2,5))

And plot them!

In [ ]:
plt.scatter(gps_data[:,2],gps_data[:,0])
plt.show()

Note that the velocities are flipped from the shapes of the plots we made before! (That is a GNSS plate reference frame issue, and nothing to be scared of $-$ we can just flip them.

In [ ]:
# let's get the velocities in a better reference frame!
v_gps=gps_data[:,0]*-1
x_gps=gps_data[:,2]

plt.scatter(x_gps,v_gps)
plt.xlabel("distance from fault (km)")  
plt.ylabel("fault-parallel velocity (mm/yr)")  
plt.show()

And we will do the rest in class!

## References


Savage, J. C., and R. O. Burford (1973), Geodetic determination of relative plate motion in central California, J. Geophys. Res., 78(5), 832–845, [doi:10.1029/JB078i005p00832](https://doi.org/10.1029/JB078i005p00832). 